In [1]:
import sys
import pandas as pd
import warnings
from tools.workflow import MultiFactorTesting,bar_resample
from pprint import pprint
import numpy as np
from tools.utility import load_pkl,save_pkl
warnings.filterwarnings('ignore')

## Notes
universe: all A share

period: 2010.01~2022.12 (156 months)

$ forward\_return = \frac{close_{next\_month}}{open_{next\_month}}-1$

$ distance =  1 - close / MAX(high,252days)$

mom3M: currentclose / close 3 months ago

turnover3M: sum of last 3 months turnover ratio

market_cap: market capitalization in circulation

bechmark for equal weighted:  equal weight of all stock's forward return

bechmark for market Cap weighted:  market Cap weight of all stock's forward return

## merged data (monthly)
处理和合并后行情和因子值的月度数据，

In [2]:
bars = ['open','close','high','low','market_cap','ep']
"""获取数据"""
# merged_data = pd.read_csv(r'data\third_data.csv')#pd.read_csv(data_file)
merged_data = load_pkl(r'data\third_data.pkl')
merged_data['date'] = pd.to_datetime(merged_data['date']) #date字段需要datetime格式；
merged_data = merged_data.set_index(['date','asset']) #必须为date,asset双重索引；
merged_data = merged_data.dropna()
merged_data.dropna().head()

merged_data['amount3M'] = merged_data.groupby(level=0).apply(lambda x:x['amount3M'].rank()/x['amount3M'].count()).droplevel(0)
merged_data['ep'] = merged_data.groupby(level=0).apply(lambda x:x['ep'].rank()/x['ep'].count()).droplevel(0)
merged_data.head()

,,close,open,high,low,avg,volume,market_cap,turnover_ratio,money,ep,skew,distance,mom1M,mom2M,mom3M,mom4M,mom5M,mom6M,mom7M,mom8M,mom9M,mom10M,mom11M,mom12M,Mom2M,turnover3M,amount3M,stddev_diff
date,asset,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2010-01-31,000001.XSHE,848.17,958.39,960.73,805.17,858.33,24294088.0,634.5328,0.8231,2.104772e+10,0.519724,0.232038,0.188175,-0.109561,0.004535,0.079190,0.123435,0.100060,-0.305198,0.199810,0.223107,0.093137,0.023835,0.155064,0.185577,-0.105523,2.5215,0.987477,0.006288
2010-02-28,000001.XSHE,877.48,848.56,884.13,820.81,875.92,10662390.0,656.4637,1.5397,9.158631e+09,0.507519,0.886190,0.160121,0.034557,-0.109561,0.004535,0.079190,0.123435,0.100060,-0.305198,0.199810,0.223107,0.093137,0.023835,0.155064,-0.078790,3.0973,0.989348,-0.005327
2010-03-31,000001.XSHE,906.80,877.48,948.62,866.54,908.75,15706730.0,678.3945,1.0449,1.428390e+10,0.988701,0.091379,0.132058,0.033414,0.034557,-0.109561,0.004535,0.079190,0.123435,0.100060,-0.305198,0.199810,0.223107,0.093137,0.023835,0.069125,3.4077,0.993095,-0.003939
2010-04-30,000001.XSHE,803.61,909.53,932.59,754.75,798.92,20433608.0,601.1979,1.1092,1.694557e+10,0.986817,-1.859854,0.230826,-0.113796,0.033414,0.034557,-0.109561,0.004535,0.079190,0.123435,0.100060,-0.305198,0.199810,0.223107,0.093137,-0.084184,3.6938,0.992467,0.006401
2010-05-31,000001.XSHE,684.40,785.63,793.45,665.24,694.56,18574160.0,512.0124,0.9249,1.332143e+10,0.991201,-0.454606,0.344928,-0.148343,-0.113796,0.033414,0.034557,-0.109561,0.004535,0.079190,0.123435,0.100060,-0.305198,0.199810,0.223107,-0.245258,3.0790,0.988686,0.006371


## forward return & factor quantile

In [3]:
mft = MultiFactorTesting(bar_data=merged_data.dropna()[bars],factor_data=merged_data.dropna()[['distance','skew']],add_shift=0)
clean_factor_return = mft.get_clean_factor_return(quantiles=5,labels={'distance':['Near','Moderate Near','Med','Moderate Far','Far'],'skew':['Low','Moderate Low','Medium','Moderate High','High']})#clean_factor_return = mft.get_clean_factor_return(quantiles={'distance':3,'skew':3})

#因子分组和下期收益数据示例
clean_factor_return.head()

next_return      distance           skew
date       asset                                                
2010-01-31 000001.XSHE     0.034081  Moderate Far  Moderate High
           000002.XSHE     0.013977           Far   Moderate Low
           000004.XSHE     0.035992          Near         Medium
           000005.XSHE     0.035326           Far           High
           000006.XSHE     0.040107           Far  Moderate High

# double sort performance
avg.ep: 当前cell成分股票平均earning/price ratio的分位数，譬如0.44表示当前cell的平均ep水平在全市场44%分位附近；

avg.liquidity: 当前cell成分股票平均近三个月成交额中位数的分位数，释义同上；

- Part I skew & distance (5x5)

In [4]:
slected_factor = ['skew','distance']
labels = {'distance':['Near','Moderate Near','Med','Moderate Far','Far'],'skew':['Low','Moderate Low','Medium','Moderate High','High']}

mft = MultiFactorTesting(bar_data=merged_data.dropna()[bars],factor_data=merged_data.dropna()[slected_factor],add_shift=0)
summary = mft.summary(quantiles=5,labels=labels)
clean_factor_return = mft.get_clean_factor_return(quantiles=5,labels=labels)
clean_factor_return = pd.concat([clean_factor_return,merged_data[['amount3M','ep']]],axis=1).dropna()
summary['avg.ep']  = round(clean_factor_return.groupby(slected_factor).apply(lambda x:x['ep'].mean()).unstack(),4)
summary['avg.liquidity']  = round(clean_factor_return.groupby(slected_factor).apply(lambda x:x['amount3M'].mean()).unstack(),4)
for i in summary:
    print(i)
    pprint(summary[i])
    print('\n')

avg_annual_excess_return_eq(%)_with_mkt=13.2%
distance       Near  Moderate Near   Med  Moderate Far   Far
skew                                                        
Low            1.28           3.58  2.72          4.06  4.77
Moderate Low   4.86           2.95  2.93          3.78  6.24
Medium         3.89           0.66 -1.56          2.07  3.58
Moderate High -3.84          -5.80 -5.69         -3.88 -1.56
High          -0.75          -7.09 -8.93         -8.42 -5.70


avg_anual_excess_return_cap(%)_with_mkt=6.18%
distance       Near  Moderate Near   Med  Moderate Far   Far
skew                                                        
Low            0.73           4.41  2.97          2.51  2.64
Moderate Low   4.86           2.97  0.82          4.06  2.77
Medium         1.17           3.57 -2.95          0.94  2.16
Moderate High  0.12          -4.01 -5.26         -1.99 -0.52
High           5.53          -4.27 -4.80         -7.11 -4.63


hit_rate_eq(%)
distance        Near  Moderate Near

- Part II Mom3M & turnover3M (5x5)

In [5]:
slected_factor = ['turnover3M','Mom2M']
mft = MultiFactorTesting(bar_data=merged_data.dropna()[bars],factor_data=merged_data.dropna()[slected_factor],add_shift=0)
labels ={'Mom2M':['Low','Moderate Low','Medium','Moderate High','High'],'turnover3M':['Low','Moderate Low','Medium','Moderate High','High']}
summary = mft.summary(quantiles=5,labels=labels)
clean_factor_return = mft.get_clean_factor_return(quantiles=5,labels=labels)
clean_factor_return = pd.concat([clean_factor_return,merged_data[['amount3M','ep']]],axis=1).dropna()
summary['avg.ep']  = round(clean_factor_return.groupby(slected_factor).apply(lambda x:x['ep'].mean()).unstack(),4)
summary['avg.liquidity']  = round(clean_factor_return.groupby(slected_factor).apply(lambda x:x['amount3M'].mean()).unstack(),4)
for i in summary:
    print(i)
    pprint(summary[i])
    print('\n')

avg_annual_excess_return_eq(%)_with_mkt=13.2%
Mom2M            Low  Moderate Low  Medium  Moderate High   High
turnover3M                                                      
Low            10.81          6.11    3.80           2.33  -0.01
Moderate Low    4.41          4.15    3.60           0.82  -0.55
Medium          3.18          2.28    2.15           0.03  -4.31
Moderate High   3.73          0.60   -0.13           0.81  -5.96
High           -0.21         -2.55   -1.95          -8.00 -20.04


avg_anual_excess_return_cap(%)_with_mkt=6.18%
Mom2M           Low  Moderate Low  Medium  Moderate High   High
turnover3M                                                     
Low            4.95          6.42    3.81           0.93   1.81
Moderate Low   5.51          7.30    0.20           2.05   2.20
Medium         5.84          4.72    0.05           3.44  -2.03
Moderate High  4.45          0.26   -1.11          -0.76  -5.97
High           1.23         -0.47   -0.98          -6.86 -14.56


h

- Part III turnover3M & distance (5x5)

In [6]:
slected_factor = ['turnover3M','distance']
mft = MultiFactorTesting(bar_data=merged_data.dropna()[bars],factor_data=merged_data.dropna()[slected_factor],add_shift=0)
labels ={'turnover3M':['Low','Moderate Low','Medium','Moderate High','High'],'distance':['Low','Moderate Low','Medium','Moderate High','High']}
summary = mft.summary(quantiles=5,labels=labels)
clean_factor_return = mft.get_clean_factor_return(quantiles=5,labels=labels)
clean_factor_return = pd.concat([clean_factor_return,merged_data[['amount3M','ep']]],axis=1).dropna()
summary['avg.ep']  = round(clean_factor_return.groupby(slected_factor).apply(lambda x:x['ep'].mean()).unstack(),4)
summary['avg.liquidity']  = round(clean_factor_return.groupby(slected_factor).apply(lambda x:x['amount3M'].mean()).unstack(),4)
for i in summary:
    print(i)
    pprint(summary[i])
    print('\n')

avg_annual_excess_return_eq(%)_with_mkt=13.2%
distance         Low  Moderate Low  Medium  Moderate High  High
turnover3M                                                     
Low            11.27          1.93    3.36           4.35  9.40
Moderate Low    5.45          2.36    0.74           2.49  3.36
Medium          1.92          0.50   -1.82           0.74  3.26
Moderate High   0.95         -0.08   -3.13          -1.07  0.95
High          -14.30         -9.81   -8.98          -7.00 -2.83


avg_anual_excess_return_cap(%)_with_mkt=6.18%
distance         Low  Moderate Low  Medium  Moderate High  High
turnover3M                                                     
Low             4.86         -1.40   -1.87           0.00  0.08
Moderate Low    6.88          3.90    1.67           2.55  1.40
Medium          3.79          1.84   -1.55           1.41  3.04
Moderate High  -2.13         -0.38   -4.84          -4.21  1.56
High          -11.49         -5.68   -9.27          -5.02 -2.88


hit_rate

- Part IV turnover3M & skew (5x5)

In [7]:
slected_factor = ['turnover3M','skew']
mft = MultiFactorTesting(bar_data=merged_data.dropna()[bars],factor_data=merged_data.dropna()[slected_factor],add_shift=0)
labels ={'turnover3M':['Low','Moderate Low','Medium','Moderate High','High'],'skew':['Low','Moderate Low','Medium','Moderate High','High']}
summary = mft.summary(quantiles=5,labels=labels)
clean_factor_return = mft.get_clean_factor_return(quantiles=5,labels=labels)
clean_factor_return = pd.concat([clean_factor_return,merged_data[['amount3M','ep']]],axis=1).dropna()
summary['avg.ep']  = round(clean_factor_return.groupby(slected_factor).apply(lambda x:x['ep'].mean()).unstack(),4)
summary['avg.liquidity']  = round(clean_factor_return.groupby(slected_factor).apply(lambda x:x['amount3M'].mean()).unstack(),4)
for i in summary:
    print(i)
    pprint(summary[i])
    print('\n')

avg_annual_excess_return_eq(%)_with_mkt=13.2%
skew            Low  Moderate Low  Medium  Moderate High   High
turnover3M                                                     
Low            0.98          9.72    7.47          -0.19   0.94
Moderate Low   4.40          4.53    3.56           0.69  -1.07
Medium         4.50          4.25    0.73          -1.63  -2.99
Moderate High  5.24          3.60    0.67          -3.20  -6.25
High           0.36         -3.23   -6.70         -11.06 -17.07


avg_anual_excess_return_cap(%)_with_mkt=6.18%
skew            Low  Moderate Low  Medium  Moderate High   High
turnover3M                                                     
Low           -0.29          1.47   -0.25          -0.07   4.73
Moderate Low   6.97          3.07    3.12           2.91   3.05
Medium         5.23          6.72    0.90           2.38  -1.05
Moderate High  3.04          1.47    0.35          -5.15  -3.05
High           0.91         -1.96   -8.15          -8.71 -12.31


hit_rate